In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from numpy.random import normal

In [2]:
#  weight parameters are uncertain with normal distribution and known mean, standard deviation
df = pd.read_csv('knapsack_dataset.csv')  
print(df)

     name  value  weight_mean  weight_std
0    gold    100          2.0         0.1
1  silver     90          1.4         0.4
2  bronze     70          1.1         0.3


In [3]:
knapsack_capacity = 3

In [4]:
%%latex 
Binary Knapsack problem with uncertainty in the weight of items. In this problem we assume that the weight of items are known
only after we made our decisions.
$$\max \sum\limits_{s\in S}p_s\sum\limits_{i = 1}^n c_i x_i$$
$$\sum\limits_{i = 1}^n  w^s_i x_i \leq b,\; \forall s \in S$$
$$x_i\in \{0, 1\}\;\forall i=1,\;...,n$$

<IPython.core.display.Latex object>

In [5]:
# generating scenarios:
number_of_scenarios = 102
scenarios = normal(df['weight_mean'], df['weight_std'], [number_of_scenarios, len(df)])

In [6]:
# print average of each parameter value over all scenarios
print([np.average(scenarios[:,0]),np.average(scenarios[:,1]),np.average(scenarios[:,2])])

[1.9901055846648983, 1.4412235877320891, 1.1191050466388581]


In [7]:
def init_weight_param(model, i, j):
    return scenarios[i, j]

In [8]:
def init_value_param(model, i):
    return df.loc[i, 'value']

In [9]:
def weight_limit_rule(m, j):
    return sum(m.w[j, i] * m.x[i] for i in range(len(df))) <= knapsack_capacity

In [10]:
model = pyo.ConcreteModel()

In [11]:
# instance definition
model.ins_items = pyo.RangeSet(0, len(df)-1)
model.ins_scenarios = pyo.RangeSet(0, number_of_scenarios-1)

In [12]:
# variable definition
model.x = pyo.Var(model.ins_items, domain=pyo.Binary)

In [13]:
# parameter defnition
model.w = pyo.Param(model.ins_scenarios, model.ins_items,
                    initialize=init_weight_param, within=pyo.PositiveReals)
model.v = pyo.Param(model.ins_items, initialize=init_value_param, within=pyo.PositiveReals)

In [14]:
# objective function definition
model.obj = pyo.Objective(expr = sum((1/number_of_scenarios) * sum(model.v[i] * model.x[i] for i in model.ins_items)
                          for _ in model.ins_scenarios), sense=pyo.maximize)

In [15]:
# constraints definition
model.weight_limit_constraints = pyo.Constraint(model.ins_scenarios, rule=weight_limit_rule)

In [16]:
# solving the model
opt = pyo.SolverFactory('cplex')
result = opt.solve(model)

In [17]:
print(model.display())

Model unknown

  Variables:
    x : Size=3, Index=ins_items
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 100.00000000000001

  Constraints:
    weight_limit_constraints : Size=102
        Key : Lower : Body               : Upper
          0 :  None : 1.9486633898231687 :   3.0
          1 :  None : 1.8909703514087597 :   3.0
          2 :  None : 2.0328119868212964 :   3.0
          3 :  None : 1.9524307244275687 :   3.0
          4 :  None :   2.09298386069052 :   3.0
          5 :  None :  1.731073504289019 :   3.0
          6 :  None : 1.9207484106429784 :   3.0
          7 :  None : 2.0507066088282877 :   3.0
          8 :  None : 2.0472316566517748 :   3.0
          9 :  None 

In [18]:
print(result)


Problem: 
- Name: tmpf30_oo05
  Lower bound: 100.00000000000001
  Upper bound: 100.00000000000001
  Number of objectives: 1
  Number of constraints: 103
  Number of variables: 4
  Number of nonzeros: 307
  Sense: maximize
Solver: 
- Status: ok
  User time: 0.02
  Termination condition: optimal
  Termination message: MIP - Integer optimal solution\x3a Objective = 1.0000000000e+02
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04297065734863281
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

